# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

### Downloads

In [130]:
%%capture
!pip install --upgrade gdown

In [131]:
# !gdown 1YC8NNo11uNbrGHUaCk9YSmu-GGDHgTx3
!gdown 1ptKHPYnGUIdB-tTEftEe8v9tCWmocyDX
# !gdown 1LN4nRqzqmiGzPTUfNh2HaROux5j6mwcp
!gdown 122aiwc0xlSBS-jHCNU0NEA0I0VTNKa-p

Downloading...
From: https://drive.google.com/uc?id=1ptKHPYnGUIdB-tTEftEe8v9tCWmocyDX
To: /content/bigrams.txt
100% 17.7M/17.7M [00:00<00:00, 56.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=122aiwc0xlSBS-jHCNU0NEA0I0VTNKa-p
To: /content/big.txt
100% 6.49M/6.49M [00:00<00:00, 41.4MB/s]


### Imports

In [132]:
import heapq
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
from collections import Counter
from tqdm.auto import tqdm
from random import random, sample, choices
from string import punctuation
from torchtext.data.metrics import bleu_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Config

In [198]:
class Config:
    # model parameters
    edit0_weight = 1
    edit1_weight = 1e-2
    edit2_weight = 1e-4
    unknown_word_threshold = 1e-8

    # test generation
    mutation_prob = 0.5
    edit_probs = [0.7, 0.275, 0.025]

### Read data
Read bigrams:

In [134]:
with open('bigrams.txt', 'r', encoding='cp1251') as f:
    data = f.read().split('\n')

unigrams = Counter()
bigrams = {}
for s in tqdm(data, desc='Counting n-grams'):
    c, w1, w2 = s.split('\t')
    c, w1, w2 = int(c), w1.lower(), w2.lower()
    unigrams[w1] += int(c)
    if w1 not in bigrams:
        bigrams[w1] = Counter()
    bigrams[w1][w2] += int(c)

N = sum(unigrams.values())

Counting n-grams:   0%|          | 0/1020385 [00:00<?, ?it/s]

### N-gram model

In [135]:
def unigram_prob(w: str):
    if w in unigrams:
        return unigrams[w] / N
    else:
        return Config.unknown_word_threshold

def bigram_prob(w1: str, w2: str):
    if w2 in bigrams[w1]:
        return bigrams[w1][w2] / unigrams[w1]
    else:
        return unigram_prob(w2)

### Spellchecking model (modified Norvig's solution)
Spellchecking model:

In [136]:
def unigram_correction(word, topk=5):
    best_corrections = heapq.nlargest(
        topk,
        candidates(word),
        key=lambda x: x[1] * unigram_prob(x[0]))
    best_corrections = [(w, score * unigram_prob(w)) for w, score in best_corrections]

    return best_corrections

def bigram_correction(word1, word2, topk=5):
    if word1 not in bigrams:
        return unigram_correction(word2, topk=topk)

    best_corrections = heapq.nlargest(
        topk,
        candidates(word2),
        key=lambda x: x[1] * bigram_prob(word1, x[0]))
    best_corrections = [(w, score * bigram_prob(word1, w)) for w, score in best_corrections]

    return best_corrections

def candidates(word):
    "Generate possible spelling corrections for word."
    # here, modified to return corrections with different edit distance
    # return [known([word]), known(edits1(word)), known(edits2(word))]
    if known([word]):
        yield word, Config.edit0_weight
    else:
        yield word, 1.

    edits1_cache = edits1(word)
    for w in known(edits1_cache):
        if w != word:
            yield w, Config.edit1_weight

    edits2_fast = (e2 for e1 in edits1_cache for e2 in edits1(e1))
    for w in known(edits2_fast):
        if w != word and w not in edits1_cache:
            yield w, Config.edit2_weight

def known(words):
    "The subset of `words` that appear in the unigrams."
    return set(w for w in words if w in unigrams)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

In [137]:
print(bigram_correction('hair', 'dked'))
print(bigram_correction('boy', 'dked'))

[('dyed', 0.000619370822472838), ('tied', 0.00010064775865183616), ('dye', 6.890500400010323e-05), ('died', 5.131570672471009e-05), ('does', 4.645281168546285e-05)]
[('died', 0.000856741573033708), ('did', 0.00027703651685393257), ('asked', 0.00012640449438202245), ('does', 0.00011903089887640449), ('need', 2.1373930620838102e-05)]


Spellchecking interface with beam search:

In [138]:
def iterate_spellchecks(beam: list, word):
    for words, beam_score in beam:
        for w2, w2_score in bigram_correction(words[-1], word, len(beam)):
            yield words + [w2], beam_score * w2_score

def spellchecking(words: list, num_beams=4, num_return_sequences=1, pbar=True, return_list=True):
    assert 0 < num_return_sequences <= num_beams

    beam = unigram_correction(words[0], topk=num_beams)
    beam = [([word], score) for word, score in beam]

    if pbar:
        bar = tqdm(words[1:], desc='Beam search')
    else:
        bar = words[1:]
    for word in bar:
        beam = heapq.nlargest(
            len(beam),
            iterate_spellchecks(beam, word),
            key=lambda x: x[1]
        )

    beam = sorted(beam, key=lambda x: x[1], reverse=True)
    if return_list:
        return beam[:num_return_sequences]
    else:
        return [TreebankWordDetokenizer().detokenize(seq) for seq, score in beam][:num_return_sequences]

In [139]:
example = "This header shouldn't was the first thang seen when viewing this Project Gutenberg file."
words = word_tokenize(example.lower())

print(*spellchecking(words), sep='\n')

Beam search:   0%|          | 0/15 [00:00<?, ?it/s]

(['this', 'reader', 'should', "n't", 'was', 'the', 'first', 'thing', 'seen', 'when', 'viewing', 'this', 'project', 'gutenberg', 'file', 'a'], 4.496293266345814e-10)


## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

### Dataset
I decided to use bigram dataset, to build bigrma spellchecking model. Nevertheless it is still possible to use fivegram dataset to build n-gram models with n <= 5, I found bigrams sufficient for the spellchecking task.

### edit0, edit1, edit2 and unknown words
To prioritize corrections with lower edit distance, n-gram probabilities are multiplied by the following weights:
- **edit0**: no correction is required (word is present in vocabulary) \\
    *weight = 1*
- **edit1**: 1 edit is required, previous weight is multiplied by *1e-2* \\
    *weight = 1e-2*
- **edit2**: 2 edits are required, previous weight is multiplied by *1e-2* \\
    *weight = 1e-4*
- **unknown**: this value is not multiplied by n-gram probability, it is constant \\
    *threshold = 1e-8*

Those weights are assigned according to intuitive rule that edit{n} misspelling is one rank more likely to occur than edit{n - 1} misspelling. More precise values are found during empirical experiments and evaluation.

### Beam search
Number of beams during evaluation is decided to be 4, which is sufficient both in terms of time and accuracy.

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

Read data:

In [140]:
with open('big.txt', 'r') as f:
    text = f.read()

sentences = sent_tokenize(text)
punkt = set(punctuation) | {'``', "''", }
sentences = [word_tokenize(s.lower()) for s in sentences if '\n' not in s]
sentences = [[w for w in s if w not in punkt] for s in sentences]
print(*sentences[:5], sep='\n')

['please', 'do', 'not', 'remove', 'it']
['to', 'sherlock', 'holmes', 'she', 'is', 'always', 'the', 'woman']
['i', 'have', 'seldom', 'heard', 'him', 'mention', 'her', 'under', 'any', 'other', 'name']
['in', 'his', 'eyes', 'she', 'eclipses', 'and', 'predominates', 'the', 'whole', 'of', 'her', 'sex']
['it', 'was', 'not', 'that', 'he', 'felt', 'any', 'emotion', 'akin', 'to', 'love', 'for', 'irene', 'adler']


Mutate sentences:

In [141]:
def mutate_word(word: str, edit_distance=1):
    for i in range(edit_distance):
        word = sample(edits1(word), 1)[0]

    return word

def random_mutate_word(word: str):
    edit_distance = choices([1, 2, 3], weights=Config.edit_probs)[0]
    return mutate_word(word, edit_distance=edit_distance)

def random_mutate_sentence(sentence: list):
    new_sentence = sentence.copy()
    for i in range(len(new_sentence)):
        if random() <= Config.mutation_prob:
            new_sentence[i] = random_mutate_word(new_sentence[i])

    return new_sentence

In [142]:
mutated_sentences = [random_mutate_sentence(s) for s in sentences]

<ipython-input-141-58fd85c0dfbe>:3: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  word = sample(edits1(word), 1)[0]


Norvig's solution:

In [178]:
def norvig_P(word):
    "Probability of `word`."
    return unigrams[word] / N

def norvig_correction(word):
    "Most probable spelling correction for word."
    return max(norvig_candidates(word), key=norvig_P)

def norvig_candidates(word):
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(norvig_edits2(word)) or [word])

def norvig_edits2(word):
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def norvig_spellchecking(sentence: list):
    return [norvig_correction(w) for w in sentence]

Evaluate BLEU score

In [194]:
def evaluate_bleu(raw_sentences, mutated_sentences, num_sentences=None, num_beams=4, pbar=True):
    candidate_sentences = []
    if num_sentences is not None:
        sent_idxs = list(range(len(raw_sentences)))
        sent_idxs = sample(sent_idxs, num_sentences)
        raw_sentences = [sent for i, sent in enumerate(raw_sentences) if i in sent_idxs]
        mutated_sentences = [sent for i, sent in enumerate(mutated_sentences) if i in sent_idxs]

    if pbar:
        bar = tqdm(mutated_sentences, desc='Evaluating BLEU score')
    else:
        bar = mutated_sentences
    for sent in bar:
        candidate_sentences.append(spellchecking(sent, num_beams=num_beams, pbar=False)[0][0])

    norvig_sentences = [norvig_spellchecking(sent) for sent in mutated_sentences]

    # print('Raw')
    # print(*raw_sentences[:5], sep='\n')
    # print('Mutated')
    # print(*mutated_sentences[:5], sep='\n')
    # print('Norvig')
    # print(*norvig_sentences[:5], sep='\n')
    # print('Canidates')
    # print(*candidate_sentences[:5], sep='\n')

    references = [[sent] for sent in raw_sentences]
    mutated_bleu = bleu_score(mutated_sentences, references, max_n=2, weights=[0.5, 0.5])
    candidates_bleu = bleu_score(candidate_sentences, references, max_n=2, weights=[0.5, 0.5])
    norvig_bleu = bleu_score(norvig_sentences, references, max_n=2, weights=[0.5, 0.5])
    return mutated_bleu, candidates_bleu, norvig_bleu

In [199]:
mutated_bleu, candidates_bleu, norvig_bleu = evaluate_bleu(
    sentences,
    mutated_sentences,
    num_sentences=100,
    num_beams=4
)
print(f'Raw BLEU score: {mutated_bleu:.6f}')
print(f'Norvig BLEU score: {norvig_bleu:.6f};    improvement: {(norvig_bleu - mutated_bleu):.6f}')
print(f'N-gram BLEU score: {candidates_bleu:.6f};    improvement: {(candidates_bleu - mutated_bleu):.6f}')

Evaluating BLEU score:   0%|          | 0/100 [00:00<?, ?it/s]

Raw BLEU score: 0.335853
Norvig BLEU score: 0.684987;    improvement: 0.349134
N-gram BLEU score: 0.785636;    improvement: 0.449783
